In [1]:
import pandas as pd
import sqlite3
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_marshmallow import Marshmallow
import os
from sqlalchemy import create_engine

basedir = os.path.abspath('.')
DATASET = 'bulldog'
df = pd.read_csv(f"../static/csv/{DATASET}.csv",usecols=['line','true']) 
df.shape

C:\Users\adsasmita\Anaconda3\envs\flask\lib\site-packages\flask_marshmallow\__init__.py:32: UserWarning: Flask-SQLAlchemy integration requires marshmallow-sqlalchemy to be installed.
  'Flask-SQLAlchemy integration requires '


(38256, 2)

In [2]:
df.head()

,line,true
0,POOR SKILLS: have any of u won a TI? Well fuck...,1
1,Tubuxis: Why immortal this is too good to see ...,0
2,Fauzan Wakid: Nara replace Loda for carry posi...,1
3,"Maya Gameworks: Insane, who wrote their script...",1
4,amir keizrul mudzir: alfredo marry alex durech...,0


In [3]:
con = sqlite3.connect(f"../db/{DATASET}.db")

#write the pandas dataframe to a sqlite table
df.to_sql(f'{DATASET}',con=con,if_exists='replace',index=True)

#connect to the database
c = con.cursor()

c.executescript(f'''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    
    ALTER TABLE {DATASET} RENAME TO old{DATASET};
    CREATE TABLE {DATASET} (id INTEGER PRIMARY KEY NOT NULL,
                            line TEXT,
                            true INTEGER);

    INSERT INTO {DATASET} SELECT * FROM old{DATASET};
    DROP TABLE old{DATASET};
    
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
con.close()

In [4]:
con = sqlite3.connect(f"../db/{DATASET}.db")
con.execute(f"PRAGMA table_info({DATASET})").fetchall()



[(0, 'id', 'INTEGER', 1, None, 1),
 (1, 'line', 'TEXT', 0, None, 0),
 (2, 'true', 'INTEGER', 0, None, 0)]

In [5]:
con.execute(f"SELECT * FROM {DATASET}").fetchall()

[(0, 'POOR SKILLS: have any of u won a TI? Well fuck u. hahahaha', 1),
 (1,
  'Tubuxis: Why immortal this is too good to see Rrueviewers dead on Pednet',
  0),
 (2, 'Fauzan Wakid: Nara replace Loda for carry position confirmed. Kappa', 1),
 (3, 'Maya Gameworks: Insane, who wrote their scripts? Jesus...', 1),
 (4,
  'amir keizrul mudzir: alfredo marry alex durechgginge che 3k trash dota 1 2 3',
  0),
 (5,
  'ramdangszall44: So were pretty good against bulldog because he has 0 INT.',
  0),
 (6,
  'Matthew: Playing DOTA game in dota2 and sometimes Syn sobaakket has a head for some cuddle monkaS',
  0),
 (7,
  'no one: Roy what happened to sinned? Donger... only...haar event mind when you play the hero everybody think so? :D',
  0),
 (8, 'Rama Kisara: that tinker blink is a dansgame osfrog', 1),
 (9, 'SenbeeDota2: RIP Bulldog X Helena Memes. BibleThump', 1),
 (10,
  'Dmitry Zaharov: i puck laugh at that picture till the brood in the comments who are saying. Even furion in 00:15 song?',
  0